In [1]:
import sys
import datetime

sys.path.append("..")

from utils.get_settings import parse

settings = parse("../utils")

settings

{'train': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv',
  'pickle_path': '../temp_result/train_data.pkl',
  'pickle_path_augument': '../temp_result/train_data_a.pkl'},
 'test': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv',
  'pickle_path': '../temp_result/test_data.pkl',
  'pickle_path_augument': '../temp_result/test_data_a.pkl'},
 'valid': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/valid_data.pkl',
  'pickle_path_augument': '../temp_result/valid_data_a.pkl'}}

In [2]:
model_name = "BIDAF_1024_neither"

In [3]:
import pandas as pd
import numpy as np
from joblib import dump, load

np.random.seed(seed=0)

import torch
torch.manual_seed(0)

In [4]:
train_data = pd.read_pickle(settings["train"]["pickle_path"])
valid_data = pd.read_pickle(settings["valid"]["pickle_path"])

train_data = pd.concat([train_data,valid_data],axis = 0)

test_data = pd.read_pickle(settings["test"]["pickle_path"])

In [5]:
columns = ['vector_bert_1024', 'pron_vector_bert_1024_mean','A_idx_bert','B_idx_bert','label','pron_idx_bert','neither_idx_bert','name_idx_bert']

In [6]:
X_train = train_data[columns]
y_train = np.array(train_data.label)


X_test = test_data[columns]
y_test = np.array(test_data.label)

In [7]:
from sklearn.model_selection import KFold 
from bidaf_neither import BIDAF
from sklearn.metrics import log_loss

score = []
pred_bidaf_tr = np.zeros((X_train.shape[0],3))

kf = KFold(n_splits=5) 
for train_index, test_index in kf.split(X_train):
    X_tr, X_ts = X_train.iloc[train_index], X_train.iloc[test_index,:]
    y_tr, y_ts = y_train[train_index], y_train[test_index]
    bidaf = BIDAF().fit(X_tr, y_tr)
    pred_ts = bidaf.predict_proba(X_ts)
    s = log_loss(y_ts,pred_ts)
    score.append(s)
    pred_bidaf_tr[test_index] = pred_ts

    

[BIDAF][BIDAF][BIDAF][BIDAF][BIDAF]

In [8]:
score

[0.4877736922869047,
 0.5146738368565248,
 0.5005870085877218,
 0.48886367496973765,
 0.5078981498230043]

In [9]:
np.array(score).mean()

0.4999592725047787

In [10]:
bidaf = BIDAF().fit(X_train, y_train)

pred_bidaf = bidaf.predict_proba(X_test)

dump((pred_bidaf_tr,pred_bidaf), "../ensemble/"+ model_name +'+basic.joblib') 

[BIDAF]

['../ensemble/BIDAF_1024_neither+basic.joblib']

In [11]:
log_loss(y_test,pred_bidaf)

0.49366917166517965

In [12]:
sub_df = pd.read_csv("../test_and_submit/sample_submission_stage_1.csv")
sub_df.loc[:, ['A','B','NEITHER']] = pred_bidaf
sub_df.to_csv("../test_and_submit/submission+model+"+model_name+"@"+str(datetime.datetime.now())+".csv", index=False)
sub_df.head()

,ID,A,B,NEITHER
0,development-1,0.416250,0.408283,0.175466
1,development-2,0.998456,0.000990,0.000554
2,development-3,0.009531,0.973183,0.017286
3,development-4,0.037526,0.764374,0.198100
4,development-5,0.000647,0.997016,0.002336


In [13]:
X_train = pd.concat([X_train,X_test],axis = 0)
y_train = np.concatenate([y_train,y_test],axis = 0)

bidaf = BIDAF().fit(X_train, y_train)

[BIDAF]

In [15]:
dump(bidaf, model_name +'.joblib') 

['BIDAF_1024_neither.joblib']